In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./city.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

from selenium.webdriver import ChromeOptions

option = ChromeOptions()

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument('--disable-blink-features=AutomationControlled')
option.add_argument('--disable-dev-shm-usage')
option.add_argument('--disable-gpu')
option.add_argument('--no-sandbox')
option.add_argument(UA.get_User_Agent_Selenium())
# option.add_argument('blink-settings=imagesEnabled=false')
# option.add_argument('--headless')
# option.add_extension(Proxy.get_Proxy_Selenium(''))

option.page_load_strategy = 'normal'

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.service import Service

import time

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By

# = = = = = = = = = = = = = = =

from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in [
          246 - 1,
          250 - 1,
          253 - 1,
          255 - 1,
          256 - 1,
          258 - 1]:
# for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    while not work.empty():
        a = work.get_nowait()
        
        output = pd.DataFrame()
        
        browser = Chrome(service=Service('../../../chromedriver'), options=option)
        browser.maximize_window()
        browser.get('https://www.yelp.com/search?find_desc=Auto+Services&find_loc=' + input_.loc[a, '英文名'] + '&attrs=job_auto_repair&sortby=rating')
        
        # = = = = = = = = = = = = = = =
        
        while True:
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(5)
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight - 3000);')
            time.sleep(5)
            browser.execute_script('window.scrollTo(0, -document.body.scrollHeight);')
            time.sleep(3)
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.2);')
            time.sleep(3)
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.4);')
            time.sleep(3)
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.6);')
            time.sleep(3)
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.8);')
            time.sleep(3)
            browser.execute_script('window.scrollTo(0, document.body.scrollHeight - 3000);')
            time.sleep(5)
                    
            # = = = = = = = = = = = = = = =
                    
            wait = WebDriverWait(browser, 300)
            wait.until(EC.presence_of_element_located((By.XPATH, '//main[@id="main-content"]/div/ul/li')))
            list_part = browser.find_elements(by=By.XPATH, value='//main[@id="main-content"]/div/ul/li')

            # = = = = = = = = = = = = = = =
                    
            script = '''let dict_arguments = {};
            for(let i=0; i<arguments[0].attributes.length; i++) {
                dict_arguments[arguments[0].attributes[i].name] = arguments[0].attributes[i].value;
            }

            return dict_arguments;'''

            wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="pagination__09f24__D23mv css-1gs0e31"]/button')))
            button = browser.find_element(by=By.XPATH, value='//div[@class="pagination__09f24__D23mv css-1gs0e31"]/button')

            dict_button = browser.execute_script(script, button)

            # = = = = = = = = = = = = = = =
            
            list_tab = []
            list_store = []
            tab = ''
            for part in list_part:
                list_h2 = part.find_elements(by=By.XPATH, value='./descendant::h2')
                if len(list_h2) != 0:
                    tab = list_h2[0].text.strip()

                if tab == '':
                    list_url = part.find_elements(by=By.XPATH, value='./descendant::div[@class="businessName__09f24__HG_pC css-1jq1ouh"]/descendant::a') + part.find_elements(by=By.XPATH, value='./descendant::h3[@class="businessName__09f24__HG_pC css-1p1tqzd"]/descendant::a')
                    if len(list_url) != 0:
                        list_tab.append('')
                        list_store.append(list_url[0].get_attribute('href').strip())
                elif 'Sponsored' in tab:
                    list_url = part.find_elements(by=By.XPATH, value='./descendant::div[@class="businessName__09f24__HG_pC css-1jq1ouh"]/descendant::a') + part.find_elements(by=By.XPATH, value='./descendant::h3[@class="businessName__09f24__HG_pC css-1p1tqzd"]/descendant::a')
                    if len(list_url) != 0:
                        list_tab.append('sponsored')
                        list_store.append(list_url[0].get_attribute('href').strip())
                elif 'All' in tab and 'result' in tab and 'in' in tab:
                    list_url = part.find_elements(by=By.XPATH, value='./descendant::div[@class="businessName__09f24__HG_pC css-1jq1ouh"]/descendant::a') + part.find_elements(by=By.XPATH, value='./descendant::h3[@class="businessName__09f24__HG_pC css-1p1tqzd"]/descendant::a')
                    if len(list_url) != 0:
                        list_tab.append('result')
                        list_store.append(list_url[0].get_attribute('href').strip())

            # = = = = = = = = = = = = = = =
            
            param_start = 0
            list_param = browser.current_url.split('?')[1].split('&')
            for param in list_param:
                if param.startswith('start='):
                    param_start = int(param.split('start=')[1])
            
            # = = = = = = = = = = = = = = =
            
            df_temp = pd.DataFrame({'序号':'',
                                    'City_No.': int(input_.loc[a, '排名']),
                                    'City': input_.loc[a, '英文名'],
                                    'Page': int(param_start / 10 + 1),
                                    'No.': [i+1 for i in range(len(list_tab))],
                                    'Tab': list_tab,
                                    'Url': list_store})

            output = pd.concat([output, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =
            
            if 'disabled' in dict_button:
                break
            else:
                button.click()
        
            # = = = = = = = = = = = = = = =
            
            print(input_.loc[a, '排名'] + ' - ' + input_.loc[a, '英文名'] +  ' - 页码：' + str(int(param_start / 10 + 1)) + ' - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']')
        
        
        # = = = = = = = = = = = = = = =
        
        output = output.drop_duplicates(ignore_index=True)
        output = output.sort_values(by=['City_No.', 'Page', 'No.'], ascending=[True, True, True]).reset_index(drop=True)
        output['序号'] = [i+1 for i in range(len(output))]
        output.to_excel('./menu/menu_' + input_.loc[a, '排名'] + '.xlsx', index=False)

        browser.quit()
        
        # = = = = = = = = = = = = = = =
        
list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)        
         
print('Done ~')     

总数量：259

250 - Killeen, Texas - 页码：1 - [当前时间：17:54:26]
246 - Norman, Oklahoma - 页码：1 - [当前时间：17:54:27]
253 - Portsmouth, Virginia - 页码：1 - [当前时间：17:54:32]
255 - Daly City, California - 页码：1 - [当前时间：17:54:33]
256 - Antioch, California - 页码：1 - [当前时间：17:54:34]
250 - Killeen, Texas - 页码：2 - [当前时间：17:54:58]
246 - Norman, Oklahoma - 页码：2 - [当前时间：17:54:58]
253 - Portsmouth, Virginia - 页码：2 - [当前时间：17:55:03]
255 - Daly City, California - 页码：2 - [当前时间：17:55:05]
256 - Antioch, California - 页码：2 - [当前时间：17:55:06]
250 - Killeen, Texas - 页码：3 - [当前时间：17:55:29]
246 - Norman, Oklahoma - 页码：3 - [当前时间：17:55:29]
253 - Portsmouth, Virginia - 页码：3 - [当前时间：17:55:35]
255 - Daly City, California - 页码：3 - [当前时间：17:55:36]
256 - Antioch, California - 页码：3 - [当前时间：17:55:38]
250 - Killeen, Texas - 页码：4 - [当前时间：17:56:00]
246 - Norman, Oklahoma - 页码：4 - [当前时间：17:56:01]
253 - Portsmouth, Virginia - 页码：4 - [当前时间：17:56:06]
255 - Daly City, California - 页码：4 - [当前时间：17:56:09]
256 - Antioch, California - 页码：4 - [当前时间：1